# Assignment 2: Scale-Space Blob Detection

### The goal of this assignment is to implement a Laplacian blob detector

Noel Mrowiec
10/1/2024
mrowiec3

In [2]:
# Libraries you will find useful
import numpy as np
import scipy 
import skimage
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageOps 

In [3]:
# Function to globally normalize a 2D array to the range [0, 1] 
def normalize(data):
    return (data - data.min()) / (data.max() - data.min())

In [5]:

# load each image 
dir_path = "images/"
images = []     #color images
filenames = []
for filename in os.listdir(dir_path):
    if filename.endswith('.jpg'):
        filenames.append(filename)
        img_path = os.path.join(dir_path, filename)
        img = Image.open(img_path)
        #img.show()  # This will display the image
        
        # convert images to grayscale
        img_gs = ImageOps.grayscale(img) 
        #img_gs.show()
        img_gs = np.array(img_gs).astype(float)
        img_gs = normalize(img_gs)
        images.append(img_gs)
        

print(f"Loaded {len(images)} images")

Minimum value: 0.0
Maximum value: 1.0
Minimum value: 0.0
Maximum value: 1.0
Minimum value: 0.0
Maximum value: 1.0
Minimum value: 0.0
Maximum value: 1.0
Loaded 4 images


In [13]:

# for applying the Laplacian filter
from scipy.ndimage.filters import gaussian_laplace
def lapacian_gaussian(image, N = 10):
    scale_space = np.empty((image.shape[0],image.shape[1],N))

    sigma_0 = 1
    for k in range(N):
        sigma = sigma_0 * (2 ** k)

        lap_gaus = gaussian_laplace(image, sigma, mode='nearest') 
        scale_norm_LoG = (sigma ** 2) * lap_gaus                                                      #scale normalized
        scale_space[:,:,k] = scale_norm_LoG

    return scale_space

/tmp/ipykernel_39506/805945956.py:2: DeprecationWarning: Please import `gaussian_laplace` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_laplace


In [11]:
from scipy.ndimage.filters import rank_filter

def non_max_suppression(array):
    local_max = rank_filter(array, rank=-1, size=3)

    # Only keep the pixels that are local maxima
    result = np.where(array == local_max, array, 0)
    return result

/tmp/ipykernel_39506/961410612.py:1: DeprecationWarning: Please import `rank_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import rank_filter


In [ ]:
def get_xy_and_radius(scale_space):
    

In [12]:
N = 10

for image in images:
    scale_space = lapacian_gaussian(image)
    for i in range(N):
        scale_space[:,:, i] = non_max_suppression(scale_space[:,:, i])


array([[0.        , 0.96673119, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.74677491],
       [0.        , 0.        , 0.        , 0.72175871, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.93132277, 0.        ],
       [0.70906027, 0.        , 0.64544579, 0.        , 0.        ,
        0.87428042, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.81650562,
        0.        , 0.97485336, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.97391024, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.81489943],


In [8]:





# filtering the image (two implementations)
# one that increases filter size, and one that downsamples the image
# For timing, use time.time()



# nonmaximum suppression in scale space
# you may find functions scipy.ndimage.filters.rank_filter or scipy.ndimage.filters.generic_filter useful



# To display the detected regions as circle
from matplotlib.patches import Circle
def show_all_circles(image, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.show()